In [ ]:
# input_video_path=f'/Users/jiyeong/Desktop/컴공 캡스톤/Dataset/ff++/val/fake'
# output_video_path=f'/Users/jiyeong/Desktop/컴공 캡스톤/Dataset/Grad-cam(Top)'   # 영상 output 저장하는 경로
# frame_path=f'/Users/jiyeong/Desktop/컴공 캡스톤/Dataset/Grad-cam(Top)'      # jpg output 저장하는 경로
# predictions_file_path = '/Users/jiyeong/Desktop/컴공 캡스톤/Dataset/(test)_checkpoint_1_predictions.xlsx' # 예측 후 메타 데이터

input_video_path=f'/Users/jiyeong/Desktop/컴공 캡스톤/Dataset/ff++/test/fake/FaceSwap'    # input
output_video_path=f'/Users/jiyeong/Desktop/컴공 캡스톤/Dataset/ff++/Grad-cam'   # 영상 output 저장하는 경로
frame_path=f'/Users/jiyeong/Desktop/컴공 캡스톤/Dataset/Grad-cam/frame'      # jpg output 저장하는 경로
predictions_file_path = '/Users/jiyeong/HUFS.CSE.DE-fake-it/model/checkpoints/(test)_checkpoint1_predictions.xlsx' # 예측 후 메타 데이터

checkpoint_path=f'/Users/jiyeong/HUFS.CSE.DE-fake-it/model/checkpoints'
checkpoint_name='checkpoint1'

# # TODO: 입력 프레임(.jpg)들이 있는 디렉토리 경로로 수정
# # input_dir = '/Users/jiyeong/Desktop/컴공 캡스톤/Dataset/Grad-cam/*'  
# input_dir = '/Users/jiyeong/Desktop/컴공 캡스톤/Dataset/ff++/Grad-cam/Top_jpg/'  
# # TODO: 결과 이미지를 저장할 디렉토리 경로로 수정
# output_dir_box='/Users/jiyeong/Desktop/컴공 캡스톤/Dataset/ff++/Grad-cam/there_boxxxx'
# base_path='/Users/jiyeong/Desktop/컴공 캡스톤/Dataset/ff++/Grad-cam'

In [2]:
# !pip install face_alignment

In [2]:
import cv2
import torch
import torchvision.transforms as T
import numpy as np
import torch.nn.functional as F
from torchvision import models
from torch import nn
import os
import glob
import pandas as pd
import face_alignment
from tqdm import tqdm

In [3]:
# ✅ MPS 디바이스 설정
device = torch.device("mps") if torch.backends.mps.is_available() else (
torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
)
print(f"Using device: {device}")

Using device: mps


In [4]:
# ✅ 모델 정의
class Model(nn.Module):
    def __init__(self, num_classes, latent_dim=2048, lstm_layers=1, hidden_dim=2048, bidirectional=False):
        super(Model, self).__init__()
        model = models.resnext50_32x4d(pretrained=True)
        self.model = nn.Sequential(*list(model.children())[:-2])
        self.lstm = nn.LSTM(latent_dim, hidden_dim, lstm_layers, bidirectional)
        self.relu = nn.LeakyReLU()
        self.dp = nn.Dropout(0.4)
        self.linear1 = nn.Linear(2048, num_classes)
        self.avgpool = nn.AdaptiveAvgPool2d(1)

    def forward(self, x):
        batch_size, seq_length, c, h, w = x.shape
        x = x.view(batch_size * seq_length, c, h, w)
        fmap = self.model(x)
        x = self.avgpool(fmap)
        x = x.view(batch_size, seq_length, 2048)
        x_lstm, _ = self.lstm(x, None)
        return fmap, self.dp(self.linear1(x_lstm[:, -1, :]))

In [12]:
# ✅ Grad-CAM 계산 함수
def compute_gradcam(model, input_tensor, target_class=None):
    model.eval()
    fmap = None
    grad = None

    def fw_hook(module, inp, out):
        nonlocal fmap
        fmap = out.detach()

    def bw_hook(module, grad_in, grad_out):
        nonlocal grad
        grad = grad_out[0].detach()

    last_layer = model.model[-1]
    f = last_layer.register_forward_hook(fw_hook)
    b = last_layer.register_backward_hook(bw_hook)

    input_tensor = input_tensor.to(device).unsqueeze(0).unsqueeze(0).requires_grad_(True)
    _, output = model(input_tensor)

    if target_class is None:
        target_class = output.argmax(dim=1).item()

    model.zero_grad()
    output[0, target_class].backward()

    weights = grad.mean(dim=[2, 3], keepdim=True)
    cam = (weights * fmap).sum(dim=1, keepdim=True)
    cam = F.relu(cam)
    cam = cam.squeeze().cpu().numpy()
    cam = (cam - cam.min()) / (cam.max() - cam.min() + 1e-8)
    cam = cv2.resize(cam, (input_tensor.shape[-1], input_tensor.shape[-2]))

    f.remove()
    b.remove()
    return cam

# ✅ MJPEG 처리 및 저장 함수
def process_video_and_save_frames(input_video_path, output_video_path, model, frame_dir=f'{frame_path}'):
    # 0. Set up
    # 1). 장치 확인
    device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")
    print(f"Using device: {device}")

    # 2). 이미 처리되어 저장된 영상 개수 확인
    input_path = f'{input_video_path}/*.mp4'
    video_files = glob.glob(input_path)
    already_present_count = glob.glob(output_video_path+ '/*.mp4')
    print("No of videos already present ", len(already_present_count))

    # 3) ouput 폴더 만들기
    os.makedirs(frame_dir, exist_ok=True)
    os.makedirs(output_video_path, exist_ok=True)
    top_jpg_dir = os.path.join(output_video_path, "Top_jpg")
    os.makedirs(top_jpg_dir, exist_ok=True)

    # 4) REAL/FAKE 라벨 불러오기
    df = pd.read_excel(predictions_file_path)

    for video_file in tqdm(video_files):
        result = str(df[df['Filepath'] == video_file]['label'].iloc[0])[0] + str(df[df['Filepath'] == video_file]['Prediction'].iloc[0])[0]
        f_name=f'({result})_'+video_file.split('/')[-1]
        out_path = os.path.join(output_video_path,f_name) # 영상 파일 이름 추출
        file_exists = glob.glob(out_path + "*")
        if(len(file_exists) != 0): # 이미 존재하면 pass
            print("File Already exists: " , out_path)
            continue
        
        cap = cv2.VideoCapture(video_file)
        fps = cap.get(cv2.CAP_PROP_FPS)
        w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

        # 5) ouput MP4 파일 이름 지정
        filename = os.path.basename(video_file)
        name, _ = os.path.splitext(filename)
        output_path = os.path.join(output_video_path, f"({result})_{name}.mp4")
        out = cv2.VideoWriter(output_path,cv2.VideoWriter_fourcc('M','J','P','G'), fps, (w, h))


        transform = T.Compose([
            T.ToPILImage(),
            T.Resize((112, 112)),
            T.ToTensor(),
            T.Normalize([0.485, 0.456, 0.406],
                                [0.229, 0.224, 0.225])
        ])

        model.to(device)
        model.eval()
        frame_count = 0

        frame_scores = []
        frame_images = []

        while True:
            ret, frame = cap.read()
            if not ret:
                break

            original = frame.copy()
            img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            img = transform(img).to(device)

            cam = compute_gradcam(model, img)

            score = float(np.mean(cam))  # ✅ 전체 cam의 평균값을 점수로 사용


            heatmap = cv2.applyColorMap(np.uint8(255 * cam), cv2.COLORMAP_JET)
            heatmap = cv2.resize(heatmap, (original.shape[1], original.shape[0]))
            overlay = 0.4 * heatmap + 0.6 * original
            overlay = np.clip(overlay, 0, 255).astype(np.uint8)

            frame_scores.append((frame_count, score))
            frame_images.append((frame_count, overlay))  # Grad-CAM 오버레이 결과 저장

            out.write(overlay) # 오버레이 프레임 저장

            # 프레임 저장
            frame_path = os.path.join(frame_dir, f"({result})_{name}_{frame_count:04d}.jpg")
            cv2.imwrite(frame_path, overlay)
            frame_count += 1
        
        # ✅ 가장 점수가 높은 상위 10개 프레임 저장
        frame_scores.sort(key=lambda x: x[1], reverse=True)  # 점수 기준으로 정렬
        top_10_indices = [idx for idx, score in frame_scores[:10]]  # 상위 10개 인덱스 추출

        for rank, idx in enumerate(top_10_indices):
            # 순위(rank)는 상위 10개 인덱스에 대한 순서입니다.
            img = frame_images[idx][1]  # top_10_indices에 해당하는 프레임 이미지를 가져옵니다.
            score = frame_scores[rank][1]  # 해당 프레임의 점수

            top_frame_path = os.path.join(top_jpg_dir, f"{f_name}_TOP{rank + 1}_Score{score:.4f}.jpg")
            cv2.imwrite(top_frame_path, img)   # 상위 10개 프레임 저장


        cap.release()  # 모든 프레임 저장
        out.release()  # mp4 저장
        print(f"✅ Grad-CAM 영상 저장 완료: {output_video_path}")
        print(f"✅ 프레임 이미지 {frame_count}개 저장됨: {frame_dir}/frame_XXXX.jpg")


In [13]:
model = Model(num_classes=2)
model.load_state_dict(torch.load(f"{checkpoint_path}/{checkpoint_name}.pt", map_location=device))

process_video_and_save_frames(
    input_video_path,
    output_video_path,
    model=model
)


/var/folders/nm/26yk90l550xc1ln38d9jn04r0000gn/T/ipykernel_961/3082594020.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f"{checkpoint_

Using device: mps
No of videos already present  0


FileNotFoundError: [Errno 2] No such file or directory: '/Users/jiyeong/Desktop/컴공 캡스톤/Dataset/(test)_checkpoint1_predictions.xlsx'

In [ ]:
def get_bbox(pts):
    x, y = pts[:,0], pts[:,1]
    return int(x.min()), int(y.min()), int(x.max()), int(y.max())

def roi_activation(cam, bbox):
    x1, y1, x2, y2 = bbox
    patch = cam[y1:y2, x1:x2]
    mean_val = float(patch.mean())

    if np.isnan(mean_val):
        return -1
    return mean_val

# ✅ 추가 함수 정의
def get_gradcam_peak(cam):
    y, x = np.unravel_index(np.argmax(cam), cam.shape)
    return x, y

def is_point_in_bbox(x, y, bbox):
    x1, y1, x2, y2 = bbox
    return x1 <= x <= x2 and y1 <= y <= y2

def get_center(bbox):
    x1, y1, x2, y2 = bbox
    return (x1 + x2) // 2, (y1 + y2) // 2

def find_roi_by_distance(cam, bbox_dict):
    peak_x, peak_y = get_gradcam_peak(cam)

    distances = []
    for region, bbox in bbox_dict.items():
        cx, cy = get_center(bbox)
        dist = ((peak_x - cx) ** 2 + (peak_y - cy) ** 2) ** 0.5
        distances.append((region, dist))

    # 거리순으로 정렬
    distances.sort(key=lambda x: x[1])
    closest_roi = distances[0][0]
    second_closest_roi = distances[1][0]

    return closest_roi, second_closest_roi, (peak_x, peak_y)



# TODO: 모델 클래스와 일치하도록 num_classes 설정
model = Model(num_classes=2)
model.load_state_dict(torch.load(f"{checkpoint_path}/{checkpoint_name}.pt", map_location=device))
model.to(device)
model.eval()

fa = face_alignment.FaceAlignment(
    face_alignment.LandmarksType.TWO_D,
    device=str(device)
)

os.makedirs(output_dir_box, exist_ok=True)

transform = T.Compose([
    T.ToTensor(),
    T.Resize((224, 224)),
    T.Normalize(mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225])
])

# ------------------------------
# ✅ 배치 처리 루프
# ------------------------------

result=[]
videos=glob.glob(os.path.join(input_dir, '*.jpg'))
for img_path in tqdm(videos):
    original = cv2.imread(img_path)
    rgb = cv2.cvtColor(original, cv2.COLOR_BGR2RGB)

    landmarks = fa.get_landmarks(rgb)
    if not landmarks:
        # 얼굴이 감지되지 않으면 패스
        continue
    lm = landmarks[0]

    # 부위별 박스 계산 (dlib/face_alignment 68점 기준)
    jl_bbox = get_bbox(lm[0:17])
    le_bbox = get_bbox(lm[36:42])
    re_bbox = get_bbox(lm[42:48])
    le_brow_bbox = get_bbox(lm[17:22])
    re_brow_bbox = get_bbox(lm[22:27])
    nose_bbox = get_bbox(lm[27:36])
    mouth_bbox = get_bbox(lm[48:68])

    # Grad-CAM 계산
    img_tensor = transform(rgb).to(device)
    cam = compute_gradcam(model, img_tensor)
    cam = cv2.resize(cam, (original.shape[1], original.shape[0]))

    # 부위별 활성도 채점
    scores = {
        'jawline':  roi_activation(cam, jl_bbox ),
        'left_eye':  roi_activation(cam, le_bbox),
        'right_eye': roi_activation(cam, re_bbox),
        'left_eye_brow': roi_activation(cam, le_brow_bbox),
        'right_eye_brow': roi_activation(cam, re_brow_bbox),
        'nose':      roi_activation(cam, nose_bbox),
        'mouth':     roi_activation(cam, mouth_bbox)
    }
    # most_activated = max(scores, key=scores.get)
    sorted_scores = sorted(scores.items(), key=lambda x: x[1], reverse=True)# 점수 정렬(null은 맨 뒤로)
    first_activated = sorted_scores[0][0]   # 가장 큰 key
    second_activated = sorted_scores[1][0]
    # 최고 활성 부위 박스 가져오기
    bbox_map = {
        'jawline': jl_bbox,
        'left_eye':  le_bbox,
        'right_eye': re_bbox,
        'left_eye_brow':le_brow_bbox,
        'right_eye_brow':re_brow_bbox,
        'nose':      nose_bbox,
        'mouth':     mouth_bbox
    }
    f_x1, f_y1, f_x2, f_y2 = bbox_map[first_activated]
    s_x1, s_y1, s_x2, s_y2 = bbox_map[second_activated]

    # Grad-CAM peak 좌표 및 가장 가까운 ROI 두 개
    peak_roi, second_peak_roi, (px, py) = find_roi_by_distance(cam, bbox_map)

    # peak가 어디 순위에 있는지 판단
    if peak_roi == first_activated:
        peak_rank = 1
    elif peak_roi == second_activated:
        peak_rank = 2
    else:
        peak_rank = -1
    
    # rank 판단은 동일하게 유지
    if second_peak_roi == first_activated:
        second_peak_rank = 1
    elif second_peak_roi == second_activated:
        second_peak_rank = 2
    else:
        second_peak_rank = -1


    # # 결과 오버레이
    # cv2.rectangle(original, (f_x1, f_y1), (f_x2, f_y2), (0, 255, 0), 2)  # Green
    # # cv2.putText(
    # #     original,
    # #     first_activated,
    # #     (f_x1, f_y1 - 10),
    # #     cv2.FONT_HERSHEY_SIMPLEX,
    # #     0.8,
    # #     (0, 255, 0),
    # #     2
    # # )

    # cv2.rectangle(original, (s_x1, s_y1), (s_x2, s_y2), (255,0, 0), 2)  # Blue
    # # cv2.putText(
    # #     original,
    # #     second_activated,
    # #     (s_x1, s_y1 - 10),
    # #     cv2.FONT_HERSHEY_SIMPLEX,
    # #     0.8,
    # #     (0, 255, 0),
    # #     2
    # # )

    result.append({
        'file_name': os.path.basename(img_path),
        'first_activate': first_activated,
        'second_activate': second_activated,
        'f_x1': f_x1, 'f_y1': f_y1, 'f_x2': f_x2, 'f_y2': f_y2,
        's_x1': s_x1, 's_y1': s_y1, 's_x2': s_x2, 's_y2': s_y2,
        'jawline': scores['jawline'],
        'left_eye': scores['left_eye'],
        'right_eye': scores['right_eye'],
        'left_eye_brow': scores['left_eye_brow'],
        'right_eye_brow': scores['right_eye_brow'],
        'nose': scores['nose'],
        'mouth': scores['mouth'],
        'gradcam_peak_x': px,
        'gradcam_peak_y': py,
        'peak_roi': peak_roi,
        'peak_rank': peak_rank,
        'peak_roi_2nd': second_peak_roi,
        'peak_rank_2nd': second_peak_rank

    })

    # 박스 친 결과 저장
    # out_path_box = os.path.join(output_dir_box, os.path.basename(img_path))
    # cv2.imwrite(out_path_box, original)

df = pd.DataFrame(result)
df = df[[
    'file_name', 'first_activate', 'second_activate',
     'peak_roi', 'peak_rank','peak_roi_2nd', 'peak_rank_2nd',
    'f_x1', 'f_y1', 'f_x2', 'f_y2',
    's_x1', 's_y1', 's_x2', 's_y2',
    'jawline','left_eye','right_eye',
    'left_eye_brow','right_eye_brow',
    'nose','mouth',
    'gradcam_peak_x', 'gradcam_peak_y'
]]

# 엑셀 저장
df.to_excel(f"{base_path}/activation_results2.xlsx", index=False)

# JSON 저장
df.to_json(f"{base_path}/activation_results2.json", orient='records', force_ascii=False)


/Users/jiyeong/anaconda3/envs/ba2023/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/jiyeong/anaconda3/envs/ba2023/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNeXt50_32X4D_Weights.IMAGENET1K_V1`. You can also use `weights=ResNeXt50_32X4D_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/var/folders/nm/26yk90l550xc1ln38d9jn04r0000gn/T/ipykernel_45950/3133928690.py:47: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code 